In [ ]:
# Note: The openai-python library support for Azure OpenAI is in preview.

import os
import openai

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_ENDPOINT")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Challenge 1

response = openai.Completion.create(
  engine="textdavinci003",
  prompt="Provide a summary of the text below and focus on the technical details.\n\nI recently purchased the 18 inch LED desktop monitor and I am extremely satisfied with my purchase. The screen clarity is exceptional and the colors are vibrant and true to life. The monitor is perfect for both work and entertainment purposes, as it provides a clear and crisp display for all types of content. The delivery of the monitor was prompt and hassle-free. The packaging was secure and protected the monitor from any damage during transit. The installation support provided by the manufacturer was also excellent. The instructions were clear and easy to follow, and I was able to set up the monitor in no time.",
  temperature=0.3,
  max_tokens=250,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response.choices[0].text)